# Securities Master Database
Aim of the code here is to maintain a database of OHLC data from different sources. Will explore tick level data in the future. Storage in google drive in csv format

In [1]:
import yfinance as yf
import pandas as pd
from google.colab import drive
import os
from datetime import datetime
import pytz
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Function to load existing tickers from log.csv
def load_existing_tickers(log_file):
    if not os.path.exists(log_file):
        return dict()
    existing_tickers = dict()
    with open(log_file, 'r') as file:
        lines = file.readlines()
        for line in lines[1:]:  # Skip the header line
            ticker, last_updated = line.strip().split(',')
            existing_tickers[ticker] = datetime.strptime(last_updated, '%Y-%m-%d')
    return existing_tickers #Return tickers: last_udpated date in dictionary form

# Function to update log.csv with new tickers and new updated dates
def update_log(log_file, new_ticker, last_updated):
    with open(log_file, 'a') as file:
        file.write(f"{new_ticker},{last_updated}\n")

#Remove duplicates from log file, as update_log will create duplicate entries
def remove_duplicates_from_log(log_file):
    if not os.path.exists(log_file):
        return
    log_data = {}
    with open(log_file, 'r') as file:
        lines = file.readlines()
        for line in lines[1:]:
            ticker, last_updated = line.strip().split(',')
            log_data.setdefault(ticker, []).append(last_updated)

    with open(log_file, 'w') as file:
        file.write("Ticker,LastUpdated\n")
        for ticker, dates in log_data.items():
            latest_date = max(dates)
            file.write(f"{ticker},{latest_date}\n")

# seems like this function is incomplete
def convert_timezone(date, time):
  datetime_str = date.split()[0] + " " + time

  # Define the format of the datetime string
  datetime_format = "%Y-%m-%d %I:%M %p"

  # Convert the combined string to a datetime object
  datetime_obj = datetime.strptime(datetime_str, datetime_format)
  return datetime_obj

# Function to fetch and store stock price data from yahoo finance, and add the open, close time of the security
def fetch_yahoo_data(ticker_list, exchange_list, storage_folder, tracking_directory):
    log_file = os.path.join(tracking_directory, 'log.csv')
    existing_tickers = load_existing_tickers(log_file)
    trading_hours_df = pd.read_csv("/content/drive/MyDrive/findata/trading_hours.csv")
    failed = []

    for ticker, exchange in zip(ticker_list, exchange_list):
      try:
        if ticker not in existing_tickers:
            # Fetch data from Yahoo Finance
            df = yf.download(ticker, end=datetime.today().strftime('%Y-%m-%d'))

            # fetch exchange information
            exchange_info = trading_hours_df[trading_hours_df["Symbol"] == exchange]
            df = df.reset_index()
            df["Date_Open"] = df.reset_index()["Date"].apply(lambda x: convert_timezone(str(x), str(exchange_info["Open"].iloc[0])))
            df["Date_Close"] = df.reset_index()["Date"].apply(lambda x: convert_timezone(str(x), str(exchange_info["Close"].iloc[0])))
            df = df.set_index("Date")

            # Group data by year
            data_by_year = df.groupby(df.index.year)

            # Create a folder for each ticker
            ticker_directory = os.path.join(storage_folder, ticker[0], ticker)
            os.makedirs(ticker_directory, exist_ok=True)

            # Save data into separate CSV files for each year in the ticker folder
            for year, year_data in data_by_year:
                year_file_name = os.path.join(ticker_directory, f"{year}.csv")
                year_data.to_csv(year_file_name)

            # Update the log file
            update_log(log_file, ticker, datetime.today().strftime('%Y-%m-%d'))
            print(f"Data for {ticker} saved to findata folder")
        else:
            print(f"Data for {ticker} already exists. Updating...")
            # Fetch data for the existing ticker from the last updated date onwards
            last_updated = existing_tickers[ticker]
            start_date = f"{last_updated.year}-01-01"
            df = yf.download(ticker, start=start_date)

            # Group data by year
            data_by_year = df.groupby(df.index.year)

            # Create a folder for each ticker
            ticker_directory = os.path.join(storage_folder, ticker[0], ticker)
            os.makedirs(ticker_directory, exist_ok=True)

            # Save data into separate CSV files for each year in the ticker folder
            for year, year_data in data_by_year:
                year_file_name = os.path.join(ticker_directory, f"{year}.csv")
                year_data.to_csv(year_file_name)

            # Update the log file
            update_log(log_file, ticker, datetime.today().strftime('%Y-%m-%d'))
            print(f"Data for {ticker} updated from {last_updated.strftime('%Y-%m-%d')} to today.")
      except:
        failed.append(ticker)
        print(f"{ticker} failed. Skipping to next")
    pd.Series(failed).to_csv("/content/drive/MyDrive/findata/failed.csv")


In [5]:
if __name__ == "__main__":
    storage_folder = "/content/drive/MyDrive/findata/OHLC_yahoo"  # Change this to your desired storage folder
    os.makedirs(storage_folder, exist_ok=True)

    # Create a directory to store the tracking file
    tracking_directory = "/content/drive/MyDrive/findata/log"
    os.makedirs(tracking_directory, exist_ok=True)

    csv_file_path = "/content/drive/MyDrive/findata/ticker_master.csv"

    if not os.path.exists(csv_file_path):
        print("CSV file not found.")
    else:
        # Read tickers from the CSV file
        tickers = pd.read_csv(csv_file_path)
        ticker_list = tickers['Ticker_yahoo'].tolist()
        exchange_list = tickers['Exchange'].tolist()

    fetch_yahoo_data(ticker_list, exchange_list, storage_folder, tracking_directory)

    # Run the function to remove duplicates from the log file
    log_file = os.path.join(tracking_directory, 'log.csv')
    remove_duplicates_from_log(log_file)

Streaming output truncated to the last 5000 lines.
Data for JEWL already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for JEWL updated from 2023-11-05 to today.
Data for JFBR already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for JFBR updated from 2023-11-05 to today.
Data for JFBRW already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for JFBRW updated from 2023-11-05 to today.
Data for JFIN already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for JFIN updated from 2023-11-05 to today.
Data for JFU already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for JFU updated from 2023-11-05 to today.
Data for JG already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for JG updated from 2023-11-05 to today.

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['TGAAW']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")



TGAAW failed. Skipping to next
Data for TGAN already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for TGAN updated from 2023-11-06 to today.
Data for TGL already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for TGL updated from 2023-11-06 to today.
Data for TGTX already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for TGTX updated from 2023-11-06 to today.
Data for TGVC already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for TGVC updated from 2023-11-06 to today.
Data for TGVCW already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for TGVCW updated from 2023-11-06 to today.
Data for TH already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for TH updated from 2023-11-06 to today.
Data for THAR alre

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['THCPW']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")



THCPW failed. Skipping to next
Data for THFF already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for THFF updated from 2023-11-06 to today.
Data for THMO already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for THMO updated from 2023-11-06 to today.
Data for THRD already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for THRD updated from 2023-11-06 to today.
Data for THRM already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for THRM updated from 2023-11-06 to today.
Data for THRX already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for THRX updated from 2023-11-06 to today.
Data for THRY already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for THRY updated from 2023-11-06 to today.
Data for THTX 

...


ZIVOW failed. Skipping to next
Data for ZJYL already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZJYL updated from 2023-11-06 to today.
Data for ZKIN already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZKIN updated from 2023-11-06 to today.
Data for ZLAB already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZLAB updated from 2023-11-06 to today.
Data for ZLS already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZLS updated from 2023-11-06 to today.
Data for ZLSWU already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZLSWU updated from 2023-11-06 to today.
[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZLSWW']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")



ZLSWW failed. Skipping to next
Data for ZM already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZM updated from 2023-11-06 to today.
Data for ZNTL already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZNTL updated from 2023-11-06 to today.
Data for ZS already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZS updated from 2023-11-06 to today.
Data for ZTEK already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZTEK updated from 2023-11-06 to today.
Data for ZUMZ already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZUMZ updated from 2023-11-06 to today.
Data for ZURA already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZURA updated from 2023-11-06 to today.
[*********************

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZURAW']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")



ZURAW failed. Skipping to next
Data for ZVRA already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZVRA updated from 2023-11-06 to today.
Data for ZVSA already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZVSA updated from 2023-11-06 to today.
Data for ZYME already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZYME updated from 2023-11-06 to today.
Data for ZYXI already exists. Updating...
[*********************100%%**********************]  1 of 1 completed
Data for ZYXI updated from 2023-11-06 to today.
